In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.matcher import Matcher

# model loading
nlp = spacy.load("en_core_web_md")

def find_similar_column(label, all_columns):
    similarities = [(other_column, label.similarity(nlp(other_column.lower()))) for other_column in all_columns]
    similarities.sort(key=lambda x: x[1], reverse=True)
    most_similar = similarities[0][0] if similarities[0][1] > 0.5 else None
    return most_similar


In [2]:
def perform_analytics(all_labels, df):
    insights = {}
    #cleaning
    #wrong dtypes
    for column in df.columns:
        if df[column].dtype == 'object':
            # Count the number of numeric values in the column
            numeric_count = pd.to_numeric(df[column], errors='coerce').notna().sum()
            
            # Check if 50% or more of the values can be converted to numbers
            if numeric_count / len(df) >= 0.5:
                # Convert the column to float, handling empty strings
                df[column] = pd.to_numeric(df[column].replace('', 'NaN'), errors='coerce').astype(float)
    #null values handling
    df.dropna(inplace = True)
    #duplicate data handling
    if df.duplicated().sum() is True:
        df.drop_duplicates
    #outliers handling
    for column in df.columns:
        if df[column].dtype == 'int64' or df[column].dtype == 'float64':
            mean = df[column].mean()
            std = df[column].std()
            z = (df[column]-mean)/std
            threshold = 3
            outliers = df[abs(z) > threshold]
            df = df[abs(z) <= threshold]
    #reseting dataset indexing 
    df = df.reset_index(drop=True)


    
    # data analysis
    #stage 1 analysis
    for label in all_labels:
        for column in df.columns:            
            if df[column].dtype == 'object':
           
                grouped_data = df.groupby([column, label])[label].count().unstack().fillna(0)
                
                labels = df[column].unique()
                
                
                grouped_data.index = labels
                
                distribution_dict = {
                    f'{label} distribution in {label}': {column_label: count for column_label, count in zip(grouped_data.columns, distribution_row)}
                    for label, distribution_row in zip(grouped_data.index, grouped_data.to_numpy())
                }
                insights[f'{column} distribution in {label} is '] = distribution_dict
                

            


            
            elif df[column].dtype == 'int64' or df[column].dtype == 'float64':
                #numerical analysis
                #here will be upgraded version wwe will build model to predict future values
                
                pass
            
            else:
                break
    #stage 2 analysis
    for label in all_labels:
        for column in df.columns:
            if df[column].dtype == 'int64' or df[column].dtype == 'float64':
                end = df[column].max()
                start = df[column].min()
                end = int(end)
                start = int(start)
                num_ranges = 10
                labl = []
                bin_edge = [start]
                for i in range(start,end,num_ranges):
                    x = (f"{i+1}-{i+num_ranges}")
                    labl.append(x)
                    bin_edge.append(i+num_ranges)
    
                df[column+'_bin'] = pd.cut(df[column], labels = labl, bins = bin_edge)
    
                grouped_data = df.groupby([column+'_bin', label], observed=False)[label].count().unstack().fillna(0)
    
                labels = df[column +'_bin'].unique()
    
                labels = grouped_data.index.get_level_values(0).intersection(labels)
    
                grouped_data = grouped_data.reindex(labels)
    
                distribution_dict = {
                    f'{label} distribution in {label}': {column_label: count for column_label, count in zip(grouped_data.columns, distribution_row)}
                    for label, distribution_row in zip(grouped_data.index, grouped_data.to_numpy())
                }
                insights[f"Distribution of {column} bin in {label} is "] = distribution_dict               
                                 #returning output insight adding left
    return insights

In [3]:
df = pd.read_csv("CustomerChurn.csv")
null_label = ['all']
print(df.duplicated().sum())
ip_labels = ["Churn"]
dataset_columns = df.columns.values.astype(str)
new_labels = []

for label in ip_labels:
    similar_column = find_similar_column(nlp(label.lower()), dataset_columns)
    if similar_column:
        new_labels.append(similar_column)

# Print similarities for debugging


all_labels = ip_labels + new_labels

print(all_labels)

0
['Churn', 'Churn']


C:\Users\adity\AppData\Local\Temp\ipykernel_26424\3413661268.py:10: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarities = [(other_column, label.similarity(nlp(other_column.lower()))) for other_column in all_columns]


In [4]:
#this block of code is temporary to continue development part and later need to be edited as required
if (label in df.columns.values for label in all_labels):
     result = perform_analytics(all_labels, df)
     for key, value in result.items():
         print(key, value)
    
else:
    
    result = default_analytics(null_label, df)
   
    
    
    

customerID distribution in Churn is  {'7590-VHVEG distribution in 7590-VHVEG': {'No': 1.0, 'Yes': 0.0}, '5575-GNVDE distribution in 5575-GNVDE': {'No': 1.0, 'Yes': 0.0}, '3668-QPYBK distribution in 3668-QPYBK': {'No': 0.0, 'Yes': 1.0}, '7795-CFOCW distribution in 7795-CFOCW': {'No': 0.0, 'Yes': 1.0}, '9237-HQITU distribution in 9237-HQITU': {'No': 0.0, 'Yes': 1.0}, '9305-CDSKC distribution in 9305-CDSKC': {'No': 1.0, 'Yes': 0.0}, '1452-KIOVK distribution in 1452-KIOVK': {'No': 1.0, 'Yes': 0.0}, '6713-OKOMC distribution in 6713-OKOMC': {'No': 1.0, 'Yes': 0.0}, '7892-POOKP distribution in 7892-POOKP': {'No': 1.0, 'Yes': 0.0}, '6388-TABGU distribution in 6388-TABGU': {'No': 1.0, 'Yes': 0.0}, '9763-GRSKD distribution in 9763-GRSKD': {'No': 1.0, 'Yes': 0.0}, '7469-LKBCI distribution in 7469-LKBCI': {'No': 1.0, 'Yes': 0.0}, '8091-TTVAX distribution in 8091-TTVAX': {'No': 1.0, 'Yes': 0.0}, '0280-XJGEX distribution in 0280-XJGEX': {'No': 1.0, 'Yes': 0.0}, '5129-JLPIS distribution in 5129-JLPIS